In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import sys
import tensorflow as tf
import pandas as pd

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

C:\Users\Denis\AppData\Local\conda\conda\envs\tensorflow_env_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import re

df_labelled_data_path = './data/global_df_data_labelled_20480.pkl'

y_chords = pd.read_pickle(df_labelled_data_path)['label']

enharmonic_notes = {'C#': 'Db',
                   'D#': 'Eb',
                   'F#': 'Gb',
                   'G#': 'Ab',
                   'A#': 'Bb'}

chords = ['C', 'C:min', 'C:maj7', 'C:min7',
         'C#', 'C#:min', 'C#:maj7', 'C#:min7',
        'D', 'D:min', 'D:maj7', 'D:min7',
        'D#', 'D#:min', 'D#:maj7', 'D#:min7',
        'E', 'E:min', 'E:maj7', 'E:min7',
        'F', 'F:min', 'F:maj7', 'F:min7',
        'F#', 'F#:min', 'F#:maj7', 'F#:min7',
        'G', 'G:min', 'G:maj7', 'G:min7',
        'G#', 'G#:min', 'G#:maj7', 'G#:min7',
        'A', 'A:min', 'A:maj7', 'A:min7',
        'A#', 'A#:min', 'A#:maj7', 'A#:min7',
        'B', 'B:min', 'B:maj7', 'B:min7',
        'N']

# chords = set()
# for item in y_chords:
#     chords.add(item)

chord_to_id = {}
id_to_chord = {}
for i, chord in enumerate(chords):
    chord_to_id[chord] = i
    id_to_chord[i] = chord

X_notprocessed = pd.read_pickle(df_labelled_data_path)['signal']
X = np.zeros((len(X_notprocessed),12,80,1), dtype=np.float)

for i, x_notprocessed in enumerate(X_notprocessed):
    X[i] = np.atleast_3d(x_notprocessed)
    
X_filtered = np.zeros((len(X),12,80,1), dtype=np.float)

y = []
print(len(y_chords))
ii = 0
for i, chord in enumerate(y_chords):
    for key, value in enharmonic_notes.items():
        chord = re.sub(value, key, chord)
    if chord in chords:
        X_filtered[ii] = X[i]
        y.append(chord_to_id[chord])
        ii+=1
        continue
X = np.zeros((len(y),12,80,1), dtype=np.float)
for i in range(len(y)):
    X[i] = X_filtered[i]
print(len(X), len(y))

48400
39207 39207


In [3]:
chords = set()
for item in y_chords:
    chords.add(item)
chords

{'A',
 'A/3',
 'A/4',
 'A/5',
 'A/6',
 'A/9',
 'A/b7',
 'A:(1)',
 'A:(1,2,4)',
 'A:7',
 'A:7(#9)',
 'A:7(*5,13)',
 'A:7(13)',
 'A:7/3',
 'A:7/5',
 'A:7/b7',
 'A:9',
 'A:9(11)',
 'A:aug',
 'A:aug/#5',
 'A:dim/b3',
 'A:dim/b5',
 'A:dim7',
 'A:maj(9)/3',
 'A:maj/9',
 'A:maj6',
 'A:maj7',
 'A:maj7/5',
 'A:min',
 'A:min(*5)',
 'A:min(2)',
 'A:min/4',
 'A:min/5',
 'A:min/6',
 'A:min/b3',
 'A:min/b7',
 'A:min6',
 'A:min7',
 'A:min7(*5,b6)',
 'A:min7(*b3)',
 'A:min9',
 'A:sus2',
 'A:sus4',
 'A:sus4(2)',
 'A:sus4/5',
 'Ab',
 'Ab/5',
 'Ab/7',
 'Ab/b7',
 'Ab:7',
 'Ab:aug',
 'Ab:maj(2)/2',
 'Ab:maj(9)',
 'Ab:maj6',
 'Ab:maj7',
 'Ab:maj9',
 'Ab:min',
 'Ab:min7',
 'B',
 'B/3',
 'B/5',
 'B/6',
 'B/7',
 'B:(b3,5)',
 'B:7',
 'B:7(#9)',
 'B:9',
 'B:aug',
 'B:aug/3',
 'B:dim',
 'B:dim7',
 'B:dim7/b9',
 'B:hdim7/b3',
 'B:hdim7/b7',
 'B:maj(*3)',
 'B:maj6',
 'B:maj7',
 'B:min',
 'B:min/5',
 'B:min/6',
 'B:min/7',
 'B:min/b3',
 'B:min/b7',
 'B:min7',
 'B:sus2',
 'B:sus4',
 'Bb',
 'Bb/3',
 'Bb/5',
 'Bb/b7',


In [4]:
from sklearn.utils import shuffle
shuffledX, shuffledy = shuffle(X, y)

trainRange = int(len(shuffledX) * 0.8)
validRange = int(len(shuffledX) * 0.9)
testRange = int(len(shuffledX) * 0.1)


X_train = np.array(shuffledX[:trainRange], dtype=np.float)
y_train = np.array(shuffledy[:trainRange], dtype=np.float)

X_valid = np.array(shuffledX[trainRange:validRange], dtype=np.float)
y_valid = np.array(shuffledy[trainRange:validRange], dtype=np.float)

X_test = np.array(shuffledX[testRange:], dtype=np.float)
y_test = np.array(shuffledy[testRange:], dtype=np.float)
print(y_test[1])
print(shuffledX.shape)

36.0
(39207, 12, 80, 1)


In [5]:
reset_graph()

#building a CNN using tensorflow
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError
import tensorflow as tf
import numpy as np
import random

he_init = tf.contrib.layers.variance_scaling_initializer()

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

class CNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None, 
                 height=12, width=80, channels=1, architecture=1, 
                 conv1={'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':None, 'conv1_activation':tf.nn.elu},
                 conv2={'conv2_fmaps':32, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.2, 'conv2_activation':tf.nn.elu}):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self.height = height
        self.width = width
        self.channels = channels
        self.conv1_fmaps = conv1['conv1_fmaps']
        self.conv1_ksize = conv1['conv1_ksize']
        self.conv1_stride = conv1['conv1_stride']
        self.conv1_dropout = conv1['conv1_dropout']
        self.conv1_activation = conv1['conv1_activation']
        self.conv2_fmaps = conv2['conv2_fmaps']
        self.conv2_ksize = conv2['conv2_ksize']
        self.conv2_stride = conv2['conv2_stride']
        self.conv2_dropout = conv2['conv2_dropout']
        self.conv2_activation = conv2['conv2_activation']
        self.architecture = architecture
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _cnn(self, inputs):
        with tf.name_scope("conv1"):
            conv1_fmaps = self.conv1_fmaps #filters
            conv1_ksize = self.conv1_ksize
            conv1_stride = self.conv1_stride
            conv1_activation = self.conv1_activation
            conv1_pad = "SAME"
            conv1_dropout = self.conv1_dropout
            conv1 = tf.layers.conv2d(inputs, filters=conv1_fmaps, kernel_size=conv1_ksize,
                                     strides=conv1_stride, padding=conv1_pad,
                                     activation=conv1_activation, name="conv1")
            if conv1_dropout:
                conv1 = tf.layers.dropout(conv1, conv1_dropout, training=self._training)

        with tf.name_scope("pool1"):
            pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
            
        with tf.name_scope("conv2"):
            conv2_fmaps = self.conv2_fmaps
            conv2_ksize = self.conv2_ksize
            conv2_stride = self.conv2_stride
            conv2_pad = "SAME"
            conv2_dropout = self.conv2_dropout
            conv2_activation = self.conv2_activation
            conv2 = tf.layers.conv2d(pool1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                                     strides=conv2_stride, padding=conv2_pad,
                                     activation=conv2_activation, name="conv2")
            if conv2_dropout:
                conv2 = tf.layers.dropout(conv2, conv2_dropout, training=self._training)

        pool2_fmaps = conv2_fmaps
        pool2_flat_shape = int(((self.height/2)/2) * ((self.width/2)/2) * pool2_fmaps)
        with tf.name_scope("pool2"):
            pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
            pool2_flat = tf.reshape(pool2, shape=[-1, pool2_flat_shape])
            

        return pool2_flat

    def _cnn2(self, inputs):
        with tf.name_scope("conv1"):
            conv1_fmaps = self.conv1_fmaps #filters
            conv1_ksize = self.conv1_ksize
            conv1_stride = self.conv1_stride
            conv1_activation = self.conv1_activation
            conv1_pad = "SAME"
            conv1_dropout = self.conv1_dropout
            conv1 = tf.layers.conv2d(inputs, filters=conv1_fmaps, kernel_size=conv1_ksize,
                                     strides=conv1_stride, padding=conv1_pad,
                                     activation=conv1_activation, name="conv1")
            

        with tf.name_scope("pool1"):
            pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
            if conv1_dropout:
                pool1 = tf.layers.dropout(pool1, conv1_dropout, training=self._training)
            
        with tf.name_scope("conv2"):
            conv2_fmaps = self.conv2_fmaps
            conv2_ksize = self.conv2_ksize
            conv2_stride = self.conv2_stride
            conv2_pad = "SAME"
            conv2_dropout = self.conv2_dropout
            conv2_activation = self.conv2_activation
            conv2 = tf.layers.conv2d(pool1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                                     strides=conv2_stride, padding=conv2_pad,
                                     activation=conv2_activation, name="conv2")

        pool2_fmaps = conv2_fmaps
        pool2_flat_shape = int(((self.height/2)/2) * ((self.width/2)/2) * pool2_fmaps)
        with tf.name_scope("pool2"):
            pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
            pool2_flat = tf.reshape(pool2, shape=[-1, pool2_flat_shape])
            if conv2_dropout:
                pool2_flat = tf.layers.dropout(pool2_flat, conv2_dropout, training=self._training)
            

        return pool2_flat

    def _cnn3(self, inputs):
        with tf.name_scope("conv1"):
            conv1_fmaps = self.conv1_fmaps #filters
            conv1_ksize = self.conv1_ksize
            conv1_stride = self.conv1_stride
            conv1_activation = self.conv1_activation
            conv1_pad = "SAME"
            conv1_dropout = self.conv1_dropout
            conv1 = tf.layers.conv2d(inputs, filters=conv1_fmaps, kernel_size=conv1_ksize,
                                     strides=conv1_stride, padding=conv1_pad,
                                     activation=conv1_activation, name="conv1")
            if conv1_dropout:
                conv1 = tf.layers.dropout(conv1, conv1_dropout, training=self._training)
            
        with tf.name_scope("conv2"):
            conv2_fmaps = self.conv2_fmaps
            conv2_ksize = self.conv2_ksize
            conv2_stride = self.conv2_stride
            conv2_pad = "SAME"
            conv2_dropout = self.conv2_dropout
            conv2_activation = self.conv2_activation
            conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                                     strides=conv2_stride, padding=conv2_pad,
                                     activation=conv2_activation, name="conv2")

        pool2_fmaps = conv2_fmaps
        pool2_flat_shape = int((self.height/2) * (self.width/2) * pool2_fmaps)
        with tf.name_scope("pool2"):
            pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
            pool2_flat = tf.reshape(pool2, shape=[-1, pool2_flat_shape])
            if conv2_dropout:
                pool2_flat = tf.layers.dropout(pool2_flat, conv2_dropout, training=self._training)
            

        return pool2_flat

    def _cnn4(self, inputs):
        with tf.name_scope("conv1"):
            conv1_fmaps = self.conv1_fmaps #filters
            conv1_ksize = self.conv1_ksize
            conv1_stride = self.conv1_stride
            conv1_activation = self.conv1_activation
            conv1_pad = "SAME"
            conv1_dropout = self.conv1_dropout
            conv1 = tf.layers.conv2d(inputs, filters=conv1_fmaps, kernel_size=conv1_ksize,
                                     strides=conv1_stride, padding=conv1_pad,
                                     activation=conv1_activation, name="conv1")
            if conv1_dropout:
                conv1 = tf.layers.dropout(conv1, conv1_dropout, training=self._training)

        pool2_fmaps = conv1_fmaps
        pool2_flat_shape = int((self.height/2) * (self.width/2) * pool2_fmaps)
        with tf.name_scope("pool2"):
            pool2 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
            pool2_flat = tf.reshape(pool2, shape=[-1, pool2_flat_shape])
            

        return pool2_flat

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""

        X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
        X_reshaped = tf.reshape(X, shape=[-1, self.height, self.width, self.channels])
        y = tf.placeholder(tf.int32, shape=[None], name="y")
        self._training = tf.placeholder_with_default(False, shape=[], name='training')

        if self.architecture == 1: #conv1 -> Drop -> max_pool -> conv2 -> Drop -> Max_pool
            cnn_outputs = self._cnn(X_reshaped)
        if self.architecture == 2: #conv1 -> max_pool -> Drop -> conv2 -> max_pool -> Drop
            cnn_outputs = self._cnn2(X_reshaped)
        if self.architecture == 3: #conv1 -> Drop-> conv2 -> max_pool -> Drop
            cnn_outputs = self._cnn3(X_reshaped)
        if self.architecture == 4: #conv -> Drop -> max_pool
            cnn_outputs = self._cnn4(X_reshaped)

        with tf.name_scope("dnn"):
            dnn_outputs = self._dnn(cnn_outputs)

        with tf.name_scope("output"):
            logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="output")
            Y_proba = tf.nn.softmax(logits, name="Y_proba")

        with tf.name_scope("train"):
            xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
            loss = tf.reduce_mean(xentropy)
            optimizer = tf.train.AdamOptimizer()
            training_op = optimizer.minimize(loss)

        with tf.name_scope("eval"):
            correct = tf.nn.in_top_k(logits, y, 1)
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

        with tf.name_scope("init_and_save"):
            init = tf.global_variables_initializer()
            saver = tf.train.Saver()

        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X_train, y_train, n_epochs=300, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = self.height * self.width * self.channels
        n_outputs = np.amax(y_train) + 1
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)

        print("***********")
        print(X_train.shape)
        print(y_train.shape)
        print(X_valid.shape)
        print(y_valid.shape)


        batch_size = self.batch_size
        best_loss_val = np.infty
        check_interval = 50
        checks_since_last_progress = 0
        max_checks_without_progress = 30
        best_model_params = None 
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            if X_valid is not None and y_valid is not None:
                for epoch in range(n_epochs):
                    rnd_idx = np.random.permutation(len(X_train))
                    idx = 0
                    for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
                        X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
                        X_batch_reshaped = np.reshape(X_batch,(len(X_batch), -1))
                        sess.run(self._training_op, feed_dict={self._X: X_batch_reshaped, self._y: y_batch, self._training: True})
                        if idx % check_interval == 0:
                            X_valid_reshaped = np.reshape(X_valid,(len(X_valid), -1))
                            loss_val = self._loss.eval(feed_dict={self._X: X_valid_reshaped,
                                                            self._y: y_valid})
                            if loss_val < best_loss_val:
                                best_loss_val = loss_val
                                checks_since_last_progress = 0
                                best_model_params = self._get_model_params()
                            else:
                                checks_since_last_progress += 1
                        idx += 1
                    X_batch_reshaped = np.reshape(X_batch,(len(X_batch), -1))
                    acc_train = self._accuracy.eval(feed_dict={self._X: X_batch_reshaped, self._y: y_batch})
                    X_valid_reshaped = np.reshape(X_valid,(len(X_valid), -1))
                    acc_val = self._accuracy.eval(feed_dict={self._X: X_valid_reshaped,
                                                       self._y: y_valid})
                    print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                              epoch, acc_train * 100, acc_val * 100, best_loss_val))
                    if checks_since_last_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break

                if best_model_params:
                    self._restore_model_params(best_model_params)
            else: 
                for epoch in range(n_epochs):
                    rnd_idx = np.random.permutation(len(X_train))
                    idx = 0
                    for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
                        X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
                        X_batch_reshaped = np.reshape(X_batch,(len(X_batch), -1))
                        sess.run(self._training_op, feed_dict={self._X: X_batch_reshaped, self._y: y_batch, self._training: True})

                    X_batch_reshaped = np.reshape(X_batch,(len(X_batch), -1))
                    acc_train = self._accuracy.eval(feed_dict={self._X: X_batch_reshaped, self._y: y_batch})
                    print("Epoch {}, train accuracy: {:.4f}%".format(
                              epoch, acc_train * 100))

            return self

    def predict(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            X_reshaped = np.reshape(X,(len(X), -1))
            return np.argmax(self._Y_proba.eval(feed_dict={self._X: X_reshaped}))

    def save(self, path):
        self._saver.save(self._session, path)

    def restore(self, path, n_inputs=960, n_outputs=48):
        self.close_session()

        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)

        self._session = tf.Session(graph=self._graph)
        
        self._saver.restore(self._session, path)

    def accuracy_score(self, X_test, y_test):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            X_test_reshaped = np.reshape(X_test,(len(X_test), -1))
            acc_test = self._accuracy.eval(feed_dict={self._X: X_test_reshaped, self._y: y_test})
            print("Final accuracy on test set:", acc_test)

            return acc_test

In [6]:
cnnClassifier = CNNClassifier(architecture=1, height=12, width=80)
cnnClassifier.fit(X_train, y_train, n_epochs=300, X_valid=X_valid, y_valid=y_valid)

***********
(31365, 12, 80, 1)
(31365,)
(3921, 12, 80, 1)
(3921,)
Epoch 0, train accuracy: 15.0000%, valid. accuracy: 21.7292%, valid. best loss: 2.662328
Epoch 1, train accuracy: 25.0000%, valid. accuracy: 21.2956%, valid. best loss: 2.653306
Epoch 2, train accuracy: 30.0000%, valid. accuracy: 20.6070%, valid. best loss: 2.635620
Epoch 3, train accuracy: 20.0000%, valid. accuracy: 22.1882%, valid. best loss: 2.619252
Epoch 4, train accuracy: 10.0000%, valid. accuracy: 22.3412%, valid. best loss: 2.619252
Early stopping!


CNNClassifier(activation=<function elu at 0x00000180F5ACB9D8>, architecture=1,
       batch_norm_momentum=None, batch_size=20, channels=1, conv1=None,
       conv2=None, dropout_rate=None, height=12,
       initializer=<function variance_scaling_initializer.<locals>._initializer at 0x00000180FAA8A840>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=None, width=80)

In [7]:
cnnClassifier.accuracy_score(X_test, y_test)

Final accuracy on test set: 0.24861847


0.24861847

In [ ]:
#random search implementation of CNN
class RandomSearchCNN(object):
    def __init__(self, params, k_fold=5, num_random_combinations=100):
        """GridSearch on CNN with cross_validation with k_fold = 5"""
        self.best_params = None
        self.k_fold = k_fold
        self.params = {
                        'n_hidden_layers':params['n_hidden_layers'],
                        'n_neurons':params['n_neurons'],
                        'optimizer_class':params['optimizer_class'],
                        'learning_rate':params['learning_rate'],
                        'dropout_rate':params['dropout_rate'],
                        'batch_size':params['batch_size'],
                        'activation':params['activation'],
                        'conv1':params['conv1'],
                        'conv2':params['conv2'],
                        'architecture':params['architecture']
        }

        max_indexes = [len(v) for k, v in self.params.items()]
        num_random_combinations = min(num_random_combinations, np.prod(max_indexes))

        # generate unique combinations
        combinations = set()
        while len(combinations) < num_random_combinations:
            combinations.add(tuple(
                random.randint(0, max_index - 1)
                for max_index in max_indexes))
        # make sure their order is shuffled
        # (`set` seems to sort its content)
        combinations = list(combinations)
        random.shuffle(combinations)

        self.combinations = combinations

    def fit(self, X_train, y_train, X_test=None, y_test=None, X_valid=None, y_valid=None, log_name='randomSearchCNN_results.txt'):

        scores = []
        print("testing", len(self.combinations), "combinations using kfold =", self.k_fold, ".")
        for combination in self.combinations:
            accuracy_rate = 0
            folds = self.k_fold
            if folds <= 1:
                if X_test == None or y_test == None:
                    raise ValueError("Pass the test set when using kfold = 1!")

                print("Trining CNN with parameters: " + 
                        "n_hidden_layers: %d, " % (self.params['n_hidden_layers'][combination[0]]) +
                        "n_neurons: %d, " % (self.params['n_neurons'][combination[1]]) +
                        "optimizer_class: %s, " % (self.params['optimizer_class'][combination[2]]) +
                        "learning_rate: %d, " % (self.params['learning_rate'][combination[3]]) +
                        "dropout_rate: %d, " % (self.params['dropout_rate'][combination[4]]) +
                        "batch_size: %d, " % (self.params['batch_size'][combination[5]]) +
                        "activation: %s, " % (self.params['activation'][combination[6]]) +
                        "conv1: %s, " % (self.params['conv1'][combination[7]]) +
                        "conv2: %s ." % (self.params['conv2'][combination[8]]) +
                        "architecture: %d, " % (self.params['architecture'][combination[9]])
                    )

                n_hidden_layers = self.params['n_hidden_layers'][combination[0]]
                n_neurons = self.params['n_neurons'][combination[1]]
                optimizer_class = self.params['optimizer_class'][combination[2]]
                learning_rate = self.params['learning_rate'][combination[3]]
                dropout_rate = self.params['dropout_rate'][combination[4]]
                batch_size = self.params['batch_size'][combination[5]]
                activation = self.params['activation'][combination[6]]
                conv1 = self.params['conv1'][combination[7]]
                conv2 = self.params['conv2'][combination[8]]
                architecture = self.params['architecture'][combination[9]]

                cnn = CNNClassifier(n_hidden_layers=n_hidden_layers, n_neurons=n_neurons, 
                    optimizer_class=optimizer_class,
                    learning_rate=learning_rate, batch_size=batch_size, 
                    dropout_rate=dropout_rate, activation=activation, 
                    conv1=conv1, conv2=conv2, architecture=architecture)

                cnn.fit(X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid)
                accuracy_rate += cnn.accuracy_score(X_test, y_test)
                del cnn

            else:
                try:
                    X = np.vstack([X_train, X_test])
                    y = np.append(y_train, y_test)
                    print('here')
                except:
                    X = X_train
                    y = y_train
                k_fold_samples = int(len(X) / folds)
                print("Trining CNN with parameters: " + 
                            "n_hidden_layers: %d, " % (self.params['n_hidden_layers'][combination[0]]) +
                            "n_neurons: %d, " % (self.params['n_neurons'][combination[1]]) +
                            "optimizer_class: %s, " % (self.params['optimizer_class'][combination[2]]) +
                            "learning_rate: %d, " % (self.params['learning_rate'][combination[3]]) +
                            "dropout_rate: %d, " % (self.params['dropout_rate'][combination[4]]) +
                            "batch_size: %d, " % (self.params['batch_size'][combination[5]]) +
                            "activation: %s, " % (self.params['activation'][combination[6]]) +
                            "conv1: %s, " % (self.params['conv1'][combination[7]]) +
                            "conv2: %s ." % (self.params['conv2'][combination[8]]) +
                            "architecture: %d, " % (self.params['architecture'][combination[9]])
                        )
                for k in range(folds):
                    n_hidden_layers = self.params['n_hidden_layers'][combination[0]]
                    n_neurons = self.params['n_neurons'][combination[1]]
                    optimizer_class = self.params['optimizer_class'][combination[2]]
                    learning_rate = self.params['learning_rate'][combination[3]]
                    dropout_rate = self.params['dropout_rate'][combination[4]]
                    batch_size = self.params['batch_size'][combination[5]]
                    activation = self.params['activation'][combination[6]]
                    conv1 = self.params['conv1'][combination[7]]
                    conv2 = self.params['conv2'][combination[8]]
                    architecture = self.params['architecture'][combination[9]]

                    print("Training and testing fold %i" % k)
                    range1 = k_fold_samples * ((folds - 1) - k)
                    range2 = k_fold_samples * (folds - k)

                    print("ranges:")
                    print(range1, range2)

                    X_train_step = X[:range1]
                    X_train_step = np.vstack([X_train, X[range2:]])
                    y_train_step = y[:range1]
                    y_train_step = np.append(y_train, y[range2:])

                    X_test_step = X[range1:range2]
                    y_test_step = y[range1:range2]

                    print("shapes:")
                    print(X_train_step.shape, X_test_step.shape)
                    print(y_train_step.shape, y_test_step.shape)

                    cnn = CNNClassifier(n_hidden_layers=n_hidden_layers, n_neurons=n_neurons, 
                        optimizer_class=optimizer_class,
                        learning_rate=learning_rate, batch_size=batch_size, 
                        dropout_rate=dropout_rate, activation=activation, 
                        conv1=conv1, conv2=conv2, architecture=architecture)

                    cnn.fit(X_train=X_train_step, y_train=y_train_step, X_valid=X_valid, y_valid=y_valid)
                    accuracy_rate += cnn.accuracy_score(X_test_step, y_test_step)

            final_accuracy_rate = accuracy_rate / folds
            score = {'n_hidden_layers': n_hidden_layers,
                            'n_neurons' : n_neurons,
                            'optimizer_class' : optimizer_class,
                            'learning_rate' : learning_rate,
                            'batch_size' : batch_size,
                            'activation' : activation,
                            'dropout_rate' : dropout_rate,
                            'conv1' : conv1,
                            'conv2' : conv2,
                            'architecture' : architecture,
                            'accuracy_rate' : final_accuracy_rate,
                        }
            scores.append(score)
            print("******************************************")
            print(score)
            print("******************************************")

            this_dir = os.path.dirname(os.path.abspath(__file__))
            search_dir = this_dir + '/search_results'
            if os.path.isdir(search_dir) == False:
                os.makedirs(search_dir)

            search_file = search_dir + '/' + log_name
            with open(search_file,"a") as f:
                f.write(str(score) + "\n")


        best_score = 0
        for score in scores:
            accuracy = score['accuracy_rate']
            if accuracy > best_score:
                best_score = accuracy
                self.best_params = score

        print("Best parameters:")
        print(self.best_params)

In [ ]:
import tensorflow as tf

def leaky_relu(alpha=0.01):
		def parametrized_leaky_relu(z, name=None):
			return tf.maximum(alpha * z, z, name=name)
		return parametrized_leaky_relu
    
"""
Best parameters (kfold = 5):

{'n_hidden_layers': 2, 'n_neurons': 500, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>, 
'learning_rate': 0.05, 'batch_size': 400, 
'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000203351C67B8>, 
'dropout_rate': 0.1, 'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 
'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x00000203335AC620>}, 'conv2': {'conv2_fmaps': 16, 
'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.2, 'conv2_activation': <function relu at 0x00000203335AC620>}, 
'architecture': 4, 'accuracy_rate': 0.9958441495895386}


Best parameters (kfold = 1 and cutting freq = 1000):

Best parameters:
{'n_hidden_layers': 8, 'n_neurons': 500, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>,
'learning_rate': 0.025, 'batch_size': 600,
'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001FDB8747950>,
'dropout_rate': 1e-06, 
'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x000001FDA77410D0>}, 
'conv2': {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.3, 'conv2_activation': <function relu at 0x000001FDA77410D0>}, 'architecture': 1, 'accuracy_rate': 0.98441553115844727}

Best parameters (kfold = 5 and cutting freq = 1000):

{'n_hidden_layers': 2, 'n_neurons': 150, 'optimizer_class': <class 'tensorflow.python.training.adagrad.AdagradOptimizer'>, 'learning_rate': 0.025, 'batch_size': 300,
'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000213B22AFD90>,
'dropout_rate': 0.2, 
'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.2, 'conv1_activation': <function relu at 0x00000213B6A4FA60>},
'conv2': {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.3, 'conv2_activation': <function relu at 0x00000213B6A4FA60>}, 'architecture': 4, 'accuracy_rate': 0.97958478927612302}

Best parameters (kfold = 5 and cutting freq = 500):
{'n_hidden_layers': 1, 'n_neurons': 300, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>,
'learning_rate': 0.05, 'batch_size': 600, 
'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001D24838D378>, 'dropout_rate': 1e-06, 
'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x000001D24678D950>}, 
'conv2': {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.2, 'conv2_activation': <function relu at 0x000001D24678D950>},
'architecture': 1, 'accuracy_rate': 0.9822956919670105}

Best parameters (kfold = 1 and cutting freq = 500):

Best parameters:
{'n_hidden_layers': 1, 'n_neurons': 400, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>, 'learning_rate': 0.05,
'batch_size': 200, 'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000243DAF0A598>,
'dropout_rate': 0.1,
'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x00000243D798D9D8>},
'conv2': {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.3, 'conv2_activation': <function relu at 0x00000243D798D9D8>},
'architecture': 1, 'accuracy_rate': 0.99299889802932739}


"""

params = {'n_hidden_layers': [1,2,3,4,5,6,7,8,9,10],
	'n_neurons' : [50, 150, 250, 300, 350, 400, 450, 500],
	'optimizer_class' : [tf.train.AdamOptimizer, tf.train.AdagradOptimizer],
	'learning_rate' : [0.01, 0.05, 0.1, 0.025],
	'batch_size' : [200, 250, 300, 400, 600],
	'activation' : [leaky_relu(), tf.nn.elu],
	'dropout_rate' : [0.000001, 0.1, 0.2, 0.3, 0.4],
	'conv1' : [{'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.elu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.2, 'conv1_activation':tf.nn.relu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.3, 'conv1_activation':tf.nn.relu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.elu}],
	'conv2' : [{'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.elu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.2, 'conv2_activation':tf.nn.relu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.3, 'conv2_activation':tf.nn.relu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.elu}],
    'architecture' : [1, 2, 3, 4]
}

rnd_search = RandomSearchCNN(params, k_fold=5, num_random_combinations=300)
rnd_search.fit(X_train, y_train, X_train, y_train, X_valid=X_valid, y_valid=y_valid)